In [ ]:
import numpy as np
import astropy as ap
import matplotlib.pyplot as plt
import sys
if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')
import groupcatalog as gc
import catalog_definitions as cat

In [ ]:
cat.sdss_published.postprocess()

In [ ]:
targets = cat.sdss_published.all_data.loc[cat.sdss_published.all_data['Z'] < 0.1]
targets = targets.loc[targets['quiescent'] == 0]
targets = targets.loc[targets['IS_SAT'] == 0]
targets.reset_index(drop=True, inplace=True)

In [ ]:
from dataloc import *
from astropy.table import Table
#nasa_sloan_atlas_file = SDSS_FOLDER + 'nsa_v0_1_2.fits'
nasa_sloan_atlas_file = '/export/sirocco1/tinker/SDSS_DATA/MANGA/nsa_v1_0_1.fits'
nsa_data = Table.read(nasa_sloan_atlas_file, format='fits')

In [ ]:
print(len(nsa_data))
print(nsa_data.columns)

In [ ]:
import astropy.coordinates as coord
import astropy.units as u
nsa_catalog = coord.SkyCoord(ra=nsa_data['RA']*u.degree, dec=nsa_data['DEC']*u.degree, frame='icrs')
to_match = coord.SkyCoord(ra=targets.RA.to_numpy()*u.degree, dec=targets.Dec.to_numpy()*u.degree, frame='icrs')

idx, d2d, d3d = coord.match_coordinates_sky(to_match, nsa_catalog, nthneighbor=1, storekdtree=False)

ang_distances = d2d.to(u.arcsec).value

ANGULAR_DISTANCE_MATCH = 3
matched = ang_distances < ANGULAR_DISTANCE_MATCH

np.sum(matched)/len(matched)

In [ ]:
print(len(nsa_catalog))
print(len(to_match))

In [ ]:
zeros = np.zeros(len(to_match), dtype=float)
BA50 = np.where(matched, nsa_data['PETRO_BA50'][idx], zeros)
BA90 = np.where(matched, nsa_data['SERSIC_BA'][idx], zeros)
#SERSIC_BA = np.where(matched, nsa_data['SERSIC_BA'][idx], zeros)

targets['PETRO_BA50'] = BA50
targets['SERSIC_BA'] = BA90
#targets['SERSIC_BA'] = SERSIC_BA


In [ ]:
targets_reduced = targets.loc[targets['PETRO_BA50'] > 0.0]
targets_reduced = targets_reduced.loc[targets_reduced['SERSIC_BA'] > 0.0]
print(len(targets_reduced), len(targets))

In [ ]:
plt.plot(np.log10(gc.Mhalo_labels), targets_reduced.groupby('Mh_bin').PETRO_BA50.mean(), label='PETRO_BA50')
plt.plot(np.log10(gc.Mhalo_labels), targets_reduced.groupby('Mh_bin').SERSIC_BA.mean(), label='SERSIC_BA')
#plt.plot(np.log10(gc.Mhalo_labels), targets.groupby('Mh_bin').SERSIC_BA.mean(), label='Sersic BA')
plt.xlim(10.75, 14)
plt.xlabel("log$(M_{halo})$")
plt.legend()

In [ ]:
plt.plot(np.log10(gc.Mhalo_labels), targets_reduced.groupby('Mh_bin').RA.count())
plt.yscale('log')

In [ ]:
plt.hist(np.log10(targets_reduced['M_HALO']), bins=20)
plt.yscale('log')

In [ ]:
plt.plot(np.log10(gc.mstar_labels), targets_reduced.groupby('Mstar_bin').PETRO_BA50.mean(), label='PETRO_BA50')
plt.plot(np.log10(gc.mstar_labels), targets_reduced.groupby('Mstar_bin').SERSIC_BA.mean(), label='SERSIC_BA')
plt.xlabel("log$(M_{\\star})$")
plt.legend()
plt.title("Axis Ratio vs Stellar Mass")

In [ ]:
# Get mode of PETRO_BA50 for each bin
BA50_bins = np.linspace(0,1,40)

def get_mode(x):
    return BA50_bins[np.histogram(x, bins=BA50_bins)[0].argmax()]

modes1 = targets_reduced.groupby('Mh_bin').PETRO_BA50.apply(lambda x: get_mode(x))
modes2 = targets_reduced.groupby('Mh_bin').SERSIC_BA.apply(lambda x: get_mode(x))

plt.plot(np.log10(gc.Mhalo_labels), modes1, label='PETRO_BA50')
plt.plot(np.log10(gc.Mhalo_labels), modes2, label='SERSIC_BA')
plt.xlim(10.75, 14)
plt.xlabel("log$(M_{halo})$")
plt.legend()

In [ ]:
frac_low = targets_reduced.groupby('Mh_bin').apply(lambda x: np.sum(x.PETRO_BA50 < 0.3)/len(x))
frac_low2 = targets_reduced.groupby('Mh_bin').apply(lambda x: np.sum(x.SERSIC_BA < 0.2)/len(x))

plt.plot(np.log10(gc.Mhalo_labels), frac_low, label='PETRO_BA50 < 0.3')
plt.plot(np.log10(gc.Mhalo_labels), frac_low2, label='SERSIC_BA, < 0.2')
plt.xlabel("log$(M_{halo})$")
plt.ylabel("Fraction with Low B/A")
plt.legend()
plt.yscale('log')
plt.xlim(10.75, 14)